#Data Wrangling Cheatsheet for Pandas
When I started doing data science with R, I found myself always coming back to the [dplyr cheatsheet ](https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf). I never had trouble finding the function I wanted by taking a quick glance at this. I haven't been able to find anything comparable for Pandas, so I made this to serve as a dplyr-pandas dictionary by translating the cheat sheet.

In [50]:
import pandas as pd
import numpy as np
from sklearn import datasets
from scipy.stats import rankdata
from IPython.display import display

iris = pd.read_csv('iris.csv')
iris.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#Reshaping Data

In [88]:
# tidyr::gather(cases,"year",n",2:4)
# gather columns into rows: Wide to Long
iris['ID']=iris.index
long_iris = pd.melt(iris,
                    id_vars = ['ID','Name'],
                    value_vars=list(iris.columns[:4]),
                    var_name = 'Part of Flower',
                    value_name='Measurement')

# tidyr::spread(pollution, size, amount)
# spread columns into rows: Long to Wide
long_iris.pivot(index='ID',columns='Part of Flower',values='Measurement')

# tidyr::separate(storms, date, c("y", "m", "d"))
iris[['first','second']] = iris['Name'].str.split('-', expand=True)
display(iris.head())


# tidyr::unite(data, col, ..., sep)
iris['new_name'] = iris['first']+"-"+iris['second']
# This second method is longer, but allows for combining a larger number of columns
iris['new_name2'] = iris[['first','second']].apply(lambda x: '-'.join(x), axis=1)
display(iris.head())
iris = iris.drop(['first','second', 'new_name','new_name2'], axis = 1)

# dplyr::rename(data, name = NAME)
display(iris.rename(columns = {'Name': 'name'}).head())
display(iris.rename(columns = lambda x: x.upper()).head())

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name,ID,first,second
0,5.1,3.5,1.4,0.2,Iris-setosa,0,Iris,setosa
1,4.9,3.0,1.4,0.2,Iris-setosa,1,Iris,setosa
2,4.7,3.2,1.3,0.2,Iris-setosa,2,Iris,setosa
3,4.6,3.1,1.5,0.2,Iris-setosa,3,Iris,setosa
4,5.0,3.6,1.4,0.2,Iris-setosa,4,Iris,setosa


,SepalLength,SepalWidth,PetalLength,PetalWidth,Name,ID,first,second,new_name,new_name2
0,5.1,3.5,1.4,0.2,Iris-setosa,0,Iris,setosa,Iris-setosa,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa,1,Iris,setosa,Iris-setosa,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa,2,Iris,setosa,Iris-setosa,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa,3,Iris,setosa,Iris-setosa,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa,4,Iris,setosa,Iris-setosa,Iris-setosa


,SepalLength,SepalWidth,PetalLength,PetalWidth,name,ID
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,2
3,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5.0,3.6,1.4,0.2,Iris-setosa,4


,SEPALLENGTH,SEPALWIDTH,PETALLENGTH,PETALWIDTH,NAME,ID
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,2
3,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5.0,3.6,1.4,0.2,Iris-setosa,4


#Subsetting Data

In [66]:
# dplyr::filter(iris,SepalLength>7)
iris[iris.SepalLength>7]

# dplyr::disinct(iris)
iris.drop_duplicates(inplace=False) # inplace=False is the default behaviour

# dplyr::sample_frac(iris,0.5,replace=True)
iris.sample(frac=0.5,replace=True) # See below to use a fixed numer instead of fraction

# dplyr::sample_n(iris,10,replace=True)
iris.sample(n=10,replace=True)

# dplyr::slice(iris,10:15) selecting rows by position
iris.iloc[10:15] # More generally, can use df.iloc[list_of_positions,:]

# dplyr::top_n(iris,2,SepalLength)
iris.sort('SepalLength',ascending=False).head(5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name,ID
131,7.9,3.8,6.4,2.0,Iris-virginica,131
135,7.7,3.0,6.1,2.3,Iris-virginica,135
122,7.7,2.8,6.7,2.0,Iris-virginica,122
117,7.7,3.8,6.7,2.2,Iris-virginica,117
118,7.7,2.6,6.9,2.3,Iris-virginica,118


#Subsetting Columns

In [77]:
# dplyr::select(iris,SepalWidth, PetalLength,PetalWidth)
iris[['SepalWidth','PetalLength','PetalWidth']];
# Or
iris.iloc[:,1:4];

#Summarising Data

In [85]:
# dplyr::summarise(iris,avg=mean(SepalLength))
avg=iris['SepalLength'].mean(); # Note R returns a DataFrame here, Pandas returns a float

# dplyr::summarise_each(iris,funs(mean))
iris.drop('Name',1).apply(np.mean); # Note we must drop the Name variable since it is a string

# dplyr::count(iris, Species)
iris['Name'].value_counts(); # Returns a Series

# dplyr::count(iris, Species, wt=SepalLength), weighted count (i.e. sum)
iris.groupby('Name')['SepalLength'].sum(); # Returns a Series;

#Making New Data

In [126]:
### Make New Variables
# dplyr::mutate (iris, sepal=SepalLength+SepalWidth)
iris['sepal']=iris.SepalLength+iris.SepalWidth # Pandas adds new column in place
iris=iris.drop('sepal',1) # Removing the newly added column for the rest of the notebook

# dplyr::mutate_each(iris,funs(min_rank))
iris.rank(method='min') # Pandas has special methods for many window functions
iris.drop('Name',1).apply(lambda x: rankdata(x,method='min')) # More general example using the rankdata function from scipy.stats. This function require numeric input so we drop the 'Name' column

# dplyr::transmute(iris,sepal=SepalLength+SepalWidth) Adds a new variable and drops original columns
iris['sepal']=iris.SepalLength+iris.SepalWidth; # Adds new variable
iris['sepal']; # Drops all others. Add more column names to keep a list
iris.drop('sepal',1,inplace=True); # Restoring our original dataset;

#Grouping Data

In [52]:
# dplyr::group_by(iris,Name)
iris.groupby('Name');

# dplyr::iris %>% group_by(Name) %>% summarise(...)
display(iris.groupby('Name').agg([np.sum,np.std]).head())

# dplyr::iris %>% group_by(Name) %>% mutate(...)
display(iris.groupby('Name').transform(lambda x: (x-np.mean(x))/np.std(x)).head())

SepalLength           SepalWidth           PetalLength  \
                        sum       std        sum       std         sum   
Name                                                                     
Iris-setosa           250.3  0.352490      170.9  0.381024        73.2   
Iris-versicolor       296.8  0.516171      138.5  0.313798       213.0   
Iris-virginica        329.4  0.635880      148.7  0.322497       277.6   

                          PetalWidth            
                      std        sum       std  
Name                                            
Iris-setosa      0.173511       12.2  0.107210  
Iris-versicolor  0.469911       66.3  0.197753  
Iris-virginica   0.551895      101.3  0.274650

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,0.269382,0.217394,-0.372597,-0.414578
1,-0.303771,-1.108180,-0.372597,-0.414578
2,-0.876924,-0.577951,-0.954780,-0.414578
3,-1.163501,-0.843065,0.209586,-0.414578
4,-0.017195,0.482509,-0.372597,-0.414578


#Combining Dataframes

In [110]:
iris1 = iris.iloc[:100,:]
iris2 = iris.iloc[50:,:]
# dplyr::left_join(iris1,iris2,by = 'name')
pd.merge(iris1, iris2, how = 'left', on = 'Name')
# The other joins can all be done using the pd.merge function

# dplyr::intersect(iris1,iris2)
pd.merge(iris1,iris2,how = 'inner')

# dplyr::union(iris1,iris2)
pd.merge(iris1,iris2, how='outer')

# dplyr::bind_rows(iris1,iris2)
pd.concat([iris1,iris2.set_index(iris1.index)],axis = 1)

# dplyr::bind_cols(iris1,iris2)
pd.concat([iris1,iris2]).reset_index();

